# Identify Updated Endpoints
**Author**:  Kena Vyas <br>
**Date**:  23rd Feb 2024 <br>
**Data Scope**: First four datasets <br>
**Report Type**: Recurring daily <br>

## Purpose
This Report gets a list of endpoints that have updated along with the entity count

In [1]:
import urllib
import pandas as pd
import ipywidgets as widgets
import numpy as np

In [2]:
datasette_url = "https://datasette.planning.data.gov.uk/"
result_df = pd.DataFrame()

def get_all_endpoints(dataset):
    global result_df
    params = urllib.parse.urlencode({
        "sql": f"""
        select 
            t1.endpoint,
            t3.pipeline 
        from(
            select 
                endpoint 
            from 
                resource_endpoint 
            group by 
                endpoint 
            having 
                count(endpoint)>1
            ) t1 
            join source t2 on t1.endpoint=t2.endpoint 
            join source_pipeline t3 on t2.source=t3.source 
        where 
            t3.pipeline='{dataset}'
        """,
        "_size": "max"
    })
    url = f"{datasette_url}digital-land.csv?{params}"
    df = pd.read_csv(url)
    result_df = df
    return df
   
dataset_options = {
    "Article 4 Direction":"article-4-direction","Article 4 Direction Area": "article-4-direction-area","Conservation Area": "conservation-area","Listed Building Outline": "listed-building-outline",
    "Tree":"tree","Tree Preservation Zone": "tree-preservation-zone","Tree":"tree","Tree Preservation Order":"tree-preservation-order"
}

global dataset_dropdown
dataset_dropdown = widgets.Dropdown(
    options=dataset_options,
)

This table lists the endpoints along with its endpoint URL and the number of entities updated

In [3]:
result=pd.DataFrame()

def get_all_endpoints_result(dataset):
    global result
    result_df=get_all_endpoints(dataset)
    selected_dataset=result_df['pipeline'][0]
    get_resource={}
    for index,row in result_df.iterrows():
        resource_endpoint=''
        params = urllib.parse.urlencode({
                "sql": f"""
                select 
                    re.resource, re.endpoint, r.start_date 
                from 
                    resource_endpoint re 
                inner join 
                    resource r on re.resource=r.resource
                where 
                    re.endpoint = '{row['endpoint']}'
                """,
                "_size": "max"
            })
        url = f"{datasette_url}digital-land.csv?{params}"
        resource_endpoint = pd.read_csv(url)
    
        resource_endpoint.sort_values(by='start_date', ascending=False, inplace=True)
        resource_endpoint.reset_index(drop=True, inplace=True)
        df=resource_endpoint.head(2)
        grouped_data = df.groupby('endpoint').apply(lambda group: [dict(zip(['resource', 'start_date'], values)) for values in group[['resource', 'start_date']].values]).reset_index(name='resource_start_date_list')
        get_resource.update(dict(zip(grouped_data['endpoint'], grouped_data['resource_start_date_list'])))
    
    all_resource_count={}
    for key,list_resource in get_resource.items():
        per_resource_count=[]
        for ele in list_resource:
            params = urllib.parse.urlencode({
                    "sql": f"""
                    SELECT COUNT(*)
                    FROM ( 
                        select rowid, end_date, fact, entry_date, entry_number, resource, start_date 
                        from fact_resource 
                        where "resource" ='{ele['resource']}' group by entry_number
                         );
                    """,
                    "_size": "max"
                })
            url = f"{datasette_url}{selected_dataset}.csv?{params}"
            count = pd.read_csv(url)
            per_resource_count.append(count.iloc[0, 0])
        all_resource_count[key] = per_resource_count
    
    endpoints_with_diff_count={}
    for key,value in all_resource_count.items():
        flag = True if value[0] != value[1] else False
        if flag:
            diff=value[0]-value[1]
            if diff < 0:
                if diff==-1:
                    msg = str(abs(diff)) + ' entity deleted'
                else:
                    msg = str(abs(diff)) + ' entities deleted'
            elif diff == 1:
                msg = str(diff) + ' entity added'
            else:
                msg = str(diff) + ' entities added'
            endpoints_with_diff_count[key]=msg
    result_df = pd.DataFrame()
    
    if len(endpoints_with_diff_count)>1:
        endpoint_tuple=tuple(endpoints_with_diff_count)
        params = urllib.parse.urlencode({
            "sql": f"""
            select 
                e.endpoint, e.endpoint_url, o.name from endpoint e inner join source s
                on e.endpoint = s.endpoint
                inner join organisation o on o.organisation = replace(s.organisation, '-eng', '')
            where 
                e.endpoint in {endpoint_tuple}
            """,
            "_size": "max"
        })
    else:
        endpoint_tuple=list(endpoints_with_diff_count.keys())[0]
        params = urllib.parse.urlencode({
            "sql": f"""
            select 
                e.endpoint, e.endpoint_url, o.name from endpoint e inner join source s
                on e.endpoint = s.endpoint
                inner join organisation o on o.organisation = replace(s.organisation, '-eng', '')
            where 
                e.endpoint =='{endpoint_tuple}'
            """,
            "_size": "max"
        })
    url = f"{datasette_url}digital-land.csv?{params}"
    df = pd.read_csv(url)
    
    to_df = pd.DataFrame.from_dict(endpoints_with_diff_count, orient='index').reset_index()
    to_df = to_df.rename(columns={'index': 'endpoint',0:'update'})
    updated_endpoints = pd.merge(to_df, df, on='endpoint', how='left')
    result=updated_endpoints
    print("Dataset : ",dataset)
    return updated_endpoints

widgets.interact(get_all_endpoints_result, dataset=dataset_dropdown)
initial_dataset = dataset_dropdown.value

interactive(children=(Dropdown(description='dataset', options={'Article 4 Direction': 'article-4-direction', '…

In [4]:
result=pd.DataFrame()
datasette_url = "https://datasette.planning.data.gov.uk/"

def track_all_endpoints(dataset):
    global output_df
    result_df=get_all_endpoints(dataset)
    selected_dataset=result_df['pipeline'][0]
    get_resource={}
    
    for index,row in result_df.iterrows():
        resource_endpoint=''
        params = urllib.parse.urlencode({
                "sql": f"""
                select 
                    re.resource, re.endpoint, r.start_date 
                from 
                    resource_endpoint re 
                inner join 
                    resource r on re.resource=r.resource
                where 
                    re.endpoint = '{row['endpoint']}'
                """,
                "_size": "max"
            })
        url = f"{datasette_url}digital-land.csv?{params}"
        resource_endpoint = pd.read_csv(url)
        resource_endpoint.sort_values(by='start_date', ascending=False, inplace=True)
        resource_endpoint.reset_index(drop=True, inplace=True)
        grouped_data = resource_endpoint.groupby('endpoint').apply(lambda group: [dict(zip(['resource', 'start_date'], values)) for values in group[['resource', 'start_date']].values]).reset_index(name='resource_start_date_list')
        get_resource.update(dict(zip(grouped_data['endpoint'], grouped_data['resource_start_date_list'])))

    all_resource_count={}
    for key,list_resource in get_resource.items():
        per_resource_count={}
        for ele in list_resource:
            params = urllib.parse.urlencode({
                    "sql": f"""
                    SELECT COUNT(*)
                    FROM ( 
                        select rowid, end_date, fact, entry_date, entry_number, resource, start_date 
                        from fact_resource 
                        where "resource" ='{ele['resource']}' group by entry_number
                         );
                    """,
                    "_size": "max"
                })
            url = f"{datasette_url}{selected_dataset}.csv?{params}"
            count = pd.read_csv(url)
            if count.iloc[0, 0] != 0:
                per_resource_count[ele['start_date']]=count.iloc[0, 0]
        if per_resource_count and len(per_resource_count) > 1:
            value=list(per_resource_count.values())
            if not all(element == value[0] for element in value):
                all_resource_count[key] = per_resource_count

    dfs = []
    for endpoint, values in all_resource_count.items():
        df_endpoint = pd.DataFrame(list(values.items()), columns=['start_date', 'value'])
        df_endpoint['start_date'] = pd.to_datetime(df_endpoint['start_date'])
        df_endpoint.sort_values('start_date', inplace=True)
        df_endpoint['value_diff'] = df_endpoint['value'].diff().fillna(0).astype(int)
        df_endpoint['Endpoint'] = endpoint
        dfs.append(df_endpoint)
    result_df = pd.concat(dfs, ignore_index=True)

    endpoint_list = result_df['Endpoint'].drop_duplicates().tolist()
    lpa_endpoint=[]
    for endpoint in endpoint_list:
        params = urllib.parse.urlencode({
                "sql": f"""
                select 
                    e.endpoint, o.name 
                from 
                    endpoint e 
                    inner join source s on e.endpoint = s.endpoint
                    inner join organisation o on o.organisation = replace(s.organisation, '-eng', '')
                where 
                    e.endpoint='{endpoint}'
                """,
                "_size": "max"
            })
        url = f"{datasette_url}digital-land.csv?{params}"
        df = pd.read_csv(url)
        lpa_endpoint.append(df)
    lpa_reshaped = np.reshape(lpa_endpoint, (-1, 2))
    endpoint_df = pd.DataFrame(lpa_reshaped, columns=['Endpoint', 'Name'])
    output_df = pd.merge(result_df, endpoint_df, on='Endpoint')
    
    ordering=['Endpoint','Name','start_date','value','value_diff']
    output_df=output_df[ordering]
    print("Dataset : ",dataset)
    return output_df

widgets.interact(track_all_endpoints, dataset=dataset_dropdown)
initial_dataset = dataset_dropdown.value

interactive(children=(Dropdown(description='dataset', options={'Article 4 Direction': 'article-4-direction', '…

In [5]:
download = input("Do you want to download the result? (yes/no): ")

if download.lower() == "yes":
    output_df.to_csv("record_updates.csv", index=False)
    print("Query result downloaded as 'record_updates.csv")

Do you want to download the result? (yes/no):  no
